In [1]:
# enabling 3rd party widgets
# from google.colab import output
# output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

# interactive 3D plot
# !pip install ipympl
# %matplotlib widget

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

import time as time

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import L2
import h5py

In [3]:
colab_flag = False

FTYPE = np.float32
ITYPE = np.int32

array = np.array
float32 = np.float32
int32 = np.int32
float64 = np.float64
int64 = np.int64

In [4]:
worker_id = 0

In [5]:
if colab_flag == True:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Github/MLROM/KS/')

In [6]:
print(os.getcwd())

/home/rkaushik/Documents/Thesis/MLROM/KS


In [7]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

if colab_flag == False:
    if gpus:
        gpu_to_use = worker_id
        tf.config.set_visible_devices(gpus[gpu_to_use], 'GPU')
        logical_devices = tf.config.list_logical_devices('GPU')
        print(logical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2022-12-24 03:54:29.410195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 03:54:29.410690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 03:54:29.482968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 03:54:29.483348: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 03:54:29.483581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [8]:
from tools.misc_tools import create_data_for_RNN
from tools.ae_v5 import Autoencoder
from tools.ESN_v3 import ESN

In [9]:
# print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())
print(tf.config.list_logical_devices())
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
[LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]
2.8.1


# KS System

In [10]:
gs_idx = '001'
dir_name_gs_all = os.getcwd() + '/grid_search/gridsearch_' + gs_idx

worker_dir_list = os.listdir(dir_name_gs_all)
worker_dir_list = [dir_name_gs_all+'/'+dname for dname in worker_dir_list if os.path.isdir(dir_name_gs_all+'/'+dname)]

# worker_dir_list = [dir_name_gs_all+'/worker_{}'.format(worker_id)]

# ESN_dir_list = []
# for dname in worker_dir_list:
#     temp = os.listdir(dname)
#     temp = [dname+'/'+esndir for esndir in temp if os.path.isdir(dname+'/'+esndir)]
#     ESN_dir_list.extend(temp)

In [11]:
print(worker_dir_list)

['/home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_001/worker_0']


In [12]:
ph_mat_list = []
ESN_dir_list = []
ph_mat_shape1 = np.inf
for i in range(len(worker_dir_list)):
    worker_dir = worker_dir_list[i]
    with np.load(worker_dir+'/prediction_horizon_mat.npz', allow_pickle=True) as f:
        # print(list(f.keys()))
        # print(f['ph_mat'])
        thisworker_phmat = f['ph_mat']
        thisworker_dirlist = f['dir_list']
    # print(len(thisworker_phmat))
    # print(np.array(thisworker_phmat).shape)
    thisworker_phmat_shape1 = len(thisworker_phmat[0])
    ph_mat_shape1 = int(np.min([ph_mat_shape1, thisworker_phmat_shape1]))
    ph_mat_list.append(thisworker_phmat)
    ESN_dir_list.extend([worker_dir+'/'+dname for dname in thisworker_dirlist])    
    
ph_mat = []
for i in range(len(ph_mat_list)):
    ph_mat.extend(list(np.array(ph_mat_list[i])[:, 0:ph_mat_shape1]))
ph_mat = np.array(ph_mat)

In [13]:
ph_mean_arr = np.mean(ph_mat, axis=1)
ph_min_arr = np.min(ph_mat, axis=1)
ph_max_arr = np.max(ph_mat, axis=1)
ph_std_arr = np.std(ph_mat, axis=1)

In [14]:
searcharr = 0.5*((ph_mean_arr-ph_std_arr) + ph_min_arr)
# searcharr = ph_mean_arr
phmeanmax_idx = np.where(searcharr == np.max(searcharr))[0][0]

ESN_dir = ESN_dir_list[phmeanmax_idx]

load_file = ESN_dir + '/final_net/final_net_class_dict.txt'
with open(load_file, 'r') as f:
    lines = f.readlines()
load_dict = eval(lines[0])

ESN_layers_units = load_dict['ESN_layers_units']
omega_in = load_dict['omega_in']
sparsity = load_dict['sparsity']
rho_res = load_dict['rho_res']
usebias_Win = load_dict['usebias_Win']
alpha = load_dict['alpha']
usebias_Wout = load_dict['usebias_Wout']
degree_of_connectivity = [int((ESN_layers_units[i]-1)*(1-sparsity[i])) for i in range(len(sparsity))]

print('max ph_mean : {}\nph_std : {}'.format(ph_mean_arr[phmeanmax_idx], ph_std_arr[phmeanmax_idx]))
print(ESN_dir)

print('ESN_layers_units : {}'.format(ESN_layers_units))
print('omega_in : {}'.format(omega_in))
print('sparsity : {}'.format(sparsity))
print('degree_of_connectivity : {}'.format(degree_of_connectivity))
print('rho_res : {}'.format(rho_res))
print('usebias_Win : {}'.format(usebias_Win))
print('alpha : {}'.format(alpha))
print('usebias_Wout : {}'.format(usebias_Wout))

max ph_mean : 1.4470169726800384
ph_std : 0.4890697158342198
/home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_001/worker_0/ESN_005
ESN_layers_units : [5000]
omega_in : [1.0]
sparsity : [0.991998399679936]
degree_of_connectivity : [40]
rho_res : [0.7]
usebias_Win : [False]
alpha : [1.0]
usebias_Wout : True


In [15]:
fac1 = 1.
fac2 = 0.25
# meancontained_idx = np.where(ph_mean_arr >= fac1*(ph_mean_arr[phmeanmax_idx]-fac2*ph_std_arr[phmeanmax_idx]))[0]
meancontained_idx = np.where(searcharr >= fac1*(searcharr[phmeanmax_idx]-fac2*ph_std_arr[phmeanmax_idx]))[0]

temp = searcharr[meancontained_idx]
temp = np.argsort(temp)[::-1]

for i in range(meancontained_idx.shape[0]):
    print('{} / {}'.format(i+1, meancontained_idx.shape[0]))
    d_idx = meancontained_idx[temp[i]]
    ESN_dir = ESN_dir_list[d_idx]

    load_file = ESN_dir + '/final_net/final_net_class_dict.txt'
    with open(load_file, 'r') as f:
        lines = f.readlines()
    load_dict = eval(lines[0])

    ESN_layers_units = load_dict['ESN_layers_units']
    omega_in = load_dict['omega_in']
    sparsity = load_dict['sparsity']
    rho_res = load_dict['rho_res']
    usebias_Win = load_dict['usebias_Win']
    alpha = load_dict['alpha']
    usebias_Wout = load_dict['usebias_Wout']
    degree_of_connectivity = [int(np.round((ESN_layers_units[i]-1)*(1-sparsity[i]))) for i in range(len(sparsity))]

    loss_file = ESN_dir + '/final_net/losses.txt'
    with open(loss_file, 'r') as f:
        lines = f.readlines()
    loss_dict = eval(lines[0])
    test_loss = loss_dict['test_loss']
    
    print('ph_mean : {}\nph_std : {}\nph_min : {}\nph_max : {}\nsearcharr : {}\n'.format(
        ph_mean_arr[d_idx],
        ph_std_arr[d_idx],
        ph_mat[d_idx].min(),
        ph_mat[d_idx].max(),
        searcharr[d_idx]
    ))
    print(ESN_dir)

    print('test_loss : {}'.format(test_loss))
    print('ESN_layers_units : {}'.format(ESN_layers_units))
    print('omega_in : {}'.format(omega_in))
    print('sparsity : {}'.format(sparsity))
    print('degree_of_connectivity : {}'.format(degree_of_connectivity))
    print('rho_res : {}'.format(rho_res))
    print('usebias_Win : {}'.format(usebias_Win))
    print('alpha : {}'.format(alpha))
    print('usebias_Wout : {}'.format(usebias_Wout))
    print('-'*20+'\n\n')

1 / 10
ph_mean : 1.4470169726800384
ph_std : 0.4890697158342198
ph_min : 0.6724296923982302
ph_max : 2.8319635122156237
searcharr : 0.8151884746220244

/home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_001/worker_0/ESN_005
test_loss : 1.4479237804231963e-06
ESN_layers_units : [5000]
omega_in : [1.0]
sparsity : [0.991998399679936]
degree_of_connectivity : [40]
rho_res : [0.7]
usebias_Win : [False]
alpha : [1.0]
usebias_Wout : True
--------------------


2 / 10
ph_mean : 1.4780521892522638
ph_std : 0.4880840114964631
ph_min : 0.6207043314445203
ph_max : 3.2328350596068764
searcharr : 0.8053362546001606

/home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_001/worker_0/ESN_000
test_loss : 1.3075192721060727e-06
ESN_layers_units : [5000]
omega_in : [1.0]
sparsity : [0.995999199839968]
degree_of_connectivity : [20]
rho_res : [0.6]
usebias_Win : [False]
alpha : [1.0]
usebias_Wout : True
--------------------


3 / 10
ph_mean : 1.444689331437121
ph_std : 0.471403951730

In [20]:
def get_ESNs_with_specific_params(
        ESN_dir_list,
        alpha=None,
        rho_res=None,
        units=None,
        omega_in=None,
        deg=None):

    selected_ESN_list = []
    for i in range(len(ESN_dir_list)):
        dname = ESN_dir_list[i]
        load_file = dname + '/final_net/final_net_class_dict.txt'
        with open(load_file, 'r') as f:
            lines = f.readlines()
        load_dict = eval(lines[0])

        ESN_layers_units_thisESN = np.array(load_dict['ESN_layers_units'])
        omega_in_thisESN = np.array(load_dict['omega_in'])
        sparsity_thisESN = np.array(load_dict['sparsity'])
        rho_res_thisESN = np.array(load_dict['rho_res'])
        usebias_Win_thisESN = np.array(load_dict['usebias_Win'])
        alpha_thisESN = np.array(load_dict['alpha'])
        usebias_Wout_thisESN = np.array(load_dict['usebias_Wout'])
        degree_of_connectivity_thisESN = np.array([int(np.round((ESN_layers_units_thisESN[i]-1)*(1-sparsity_thisESN[i]))) for i in range(len(sparsity_thisESN))])

        alpha_pass = True
        if type(alpha) != type(None):
            alpha_pass = np.all(alpha == alpha_thisESN)
            
        rho_res_pass = True
        if type(rho_res) != type(None):
            rho_res_pass = np.all(rho_res == rho_res_thisESN)

        units_pass = True
        if type(units) != type(None):
            units_pass = np.all(units == ESN_layers_units_thisESN)

        omega_in_pass = True
        if type(omega_in) != type(None):
            omega_in_pass = np.all(omega_in == omega_in_thisESN)

        deg_pass = True
        if type(deg) != type(None):
            deg_pass = np.all(deg == degree_of_connectivity_thisESN)

        if alpha_pass and rho_res_pass and units_pass and omega_in_pass and deg_pass:
            # print(degree_of_connectivity_thisESN)
            selected_ESN_list.append((dname, i))
        
    return selected_ESN_list

In [21]:
selected_ESN_list = get_ESNs_with_specific_params(ESN_dir_list, units=np.array([2000]))#, deg=np.array([3]))

In [22]:
d_idx_arr = np.array([elem[1] for elem in selected_ESN_list])
temp = ph_mean_arr[d_idx_arr]
temp = np.argsort(temp)[::-1]
temp2 = [selected_ESN_list[idx] for idx in temp]
selected_ESN_list = temp2

for i in range(len(selected_ESN_list)):
    print('{} / {}'.format(i+1, len(selected_ESN_list)))
    ESN_dir = selected_ESN_list[i][0]
    d_idx = selected_ESN_list[i][1]

    load_file = ESN_dir + '/final_net/final_net_class_dict.txt'
    with open(load_file, 'r') as f:
        lines = f.readlines()
    load_dict = eval(lines[0])

    ESN_layers_units = load_dict['ESN_layers_units']
    omega_in = load_dict['omega_in']
    sparsity = load_dict['sparsity']
    rho_res = load_dict['rho_res']
    usebias_Win = load_dict['usebias_Win']
    alpha = load_dict['alpha']
    usebias_Wout = load_dict['usebias_Wout']
    degree_of_connectivity = [int(np.round((ESN_layers_units[i]-1)*(1-sparsity[i]))) for i in range(len(sparsity))]

    loss_file = ESN_dir + '/final_net/losses.txt'
    with open(loss_file, 'r') as f:
        lines = f.readlines()
    loss_dict = eval(lines[0])
    test_loss = loss_dict['test_loss']
    
    print('ph_mean : {}\nph_std : {}\nph_min : {}\nph_max : {}\nsearcharr : {}\n'.format(
        ph_mean_arr[d_idx],
        ph_std_arr[d_idx],
        ph_mat[d_idx].min(),
        ph_mat[d_idx].max(),
        searcharr[d_idx]
    ))
    print(ESN_dir)

    print('test_loss : {}'.format(test_loss))
    print('ESN_layers_units : {}'.format(ESN_layers_units))
    print('omega_in : {}'.format(omega_in))
    print('sparsity : {}'.format(sparsity))
    print('degree_of_connectivity : {}'.format(degree_of_connectivity))
    print('rho_res : {}'.format(rho_res))
    print('usebias_Win : {}'.format(usebias_Win))
    print('alpha : {}'.format(alpha))
    print('usebias_Wout : {}'.format(usebias_Wout))
    print('-'*20+'\n\n')

1 / 81
ph_mean : 0.9095904723709906
ph_std : 0.2711959490930297
ph_min : 0.3750088669143977
ph_max : 1.6034861895650108
searcharr : 0.5067016950961793

/home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_000/worker_1/ESN_061
test_loss : 1.9208654468028787e-06
ESN_layers_units : [2000]
omega_in : [0.5]
sparsity : [0.8999499749874937]
degree_of_connectivity : [200]
rho_res : [0.8]
usebias_Win : [False]
alpha : [1.0]
usebias_Wout : True
--------------------


2 / 81
ph_mean : 0.9058403837018468
ph_std : 0.3068483335527765
ph_min : 0.4525969083449627
ph_max : 1.9784950564794084
searcharr : 0.5257944792470165

/home/rkaushik/Documents/Thesis/MLROM/KS/grid_search/gridsearch_000/worker_1/ESN_073
test_loss : 1.967996700947249e-06
ESN_layers_units : [2000]
omega_in : [1.0]
sparsity : [0.8999499749874937]
degree_of_connectivity : [200]
rho_res : [0.5]
usebias_Win : [False]
alpha : [1.0]
usebias_Wout : True
--------------------


3 / 81
ph_mean : 0.8687274372175599
ph_std : 0.27575233

In [ ]:
# def return_params(fname):
#     with np.load(fname, allow_pickle=True) as fl:
#         alpha = fl['alpha']
#         deg = fl['deg_of_connectivity']
#         omega_in = fl['omega_in']
#         rho_res = fl['rho_res']
#         res_size = fl['res_size']
#     return_dict = {
#         'alpha':list(alpha),
#         'deg':list(deg),
#         'omega_in':list(omega_in),
#         'res_size':list(res_size),
#         'rho_res':list(rho_res),
#     }
#     return return_dict

In [ ]:
# dname = os.getcwd() + '/grid_search/gridsearch_000'
# fname = dname+'/gridsearch_params.npz'
# params = return_params(fname)

In [ ]:
# params

In [ ]:
# with open(fname[0:-4]+'.txt', 'w') as fl:
#     fl.write(str(params))